# Import dependencies

In [1]:
import csv
import cv2
import fnmatch
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Activation, Dense, Flatten
from keras.models import load_model, Sequential
import math
from matplotlib import pyplot
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

Using TensorFlow backend.


# Set parameters

In [2]:
CAMERA_COUNT = 3

DRIVING_LOG_PATH = './data'
DRIVING_LOG_FILE = 'driving_log.csv'

CENTER_IMAGE_REGULAR_EXPRESSION = 'center*'
IMAGE_PATH = './data/IMG'

SET_SIZE = 256
EPOCH = 4
VALIDATION_SET_SIZE = 0.2

center_file = 'center_2016_12_01_13_37_16_570.jpg'
left_file = 'left_2016_12_01_13_37_16_570.jpg'
right_file = 'right_2016_12_01_13_37_16_570.jpg'

# Get data

Use data by Udacity.

Source: https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip

Size: 322.8 MB

In [ ]:
# Do not execute this cell as it will read all images into memory
# data = np.array([cv2.imread(os.path.join(IMAGE_PATH, file)) for file in os.listdir(IMAGE_PATH)], dtype = 'float32')

# Explore data

In [ ]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

print(reader.__next__())
print(reader.__next__())

In [ ]:
count = len(os.listdir(IMAGE_PATH))

print('Count:', count)
print('Frames:', count / CAMERA_COUNT)

In [ ]:
data = np.array([1 for file in os.listdir(IMAGE_PATH) if fnmatch.fnmatch(file, CENTER_IMAGE_REGULAR_EXPRESSION)], dtype = 'float32')
print('Center image count:', data.sum())

In [ ]:
image_center = cv2.imread(os.path.join(IMAGE_PATH, center_file))
print('Shape:', image_center.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_left = cv2.imread(os.path.join(IMAGE_PATH, left_file))
pyplot.imshow(cv2.cvtColor(image_left, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_right = cv2.imread(os.path.join(IMAGE_PATH, right_file))
pyplot.imshow(cv2.cvtColor(image_right, cv2.COLOR_BGR2RGB))
pyplot.show()

# Transform data

In [ ]:
image_center_resized = cv2.resize(image_center, (32, 16))
print('Shape:', image_center_resized.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_resized, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
y_start = 60
y_end = image_center.shape[0] - 20
x_start = 0
x_end = image_center.shape[1]

image_center_cropped = image_center[y_start:y_end, x_start:x_end]
print('Shape:', image_center_cropped.shape)

In [ ]:
pyplot.imshow(cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2RGB))
pyplot.show()

In [ ]:
image_center_grayscale = cv2.cvtColor(image_center, cv2.COLOR_BGR2GRAY)

In [ ]:
pyplot.imshow(image_center_grayscale, cmap = 'gray')
pyplot.show()

In [ ]:
image_center_cropped_grayscale = cv2.cvtColor(image_center_cropped, cv2.COLOR_BGR2GRAY)
image_center_cropped_grayscale_resized = cv2.resize(image_center_cropped_grayscale, (32, 8))

In [ ]:
pyplot.imshow(image_center_cropped_grayscale_resized, cmap = 'gray')
pyplot.show()

# Get data

In [3]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

reader.__next__()

features = np.array(
    [
        cv2.imread(
            os.path.join(
                IMAGE_PATH,
                reader
                    .__next__()[0]
                    .strip('IMG/')
            )
        )
        for index in range(SET_SIZE)
    ],
    dtype = 'float32'
)

In [4]:
file = open(os.path.join(DRIVING_LOG_PATH, DRIVING_LOG_FILE), 'r')
reader = csv.reader(file)

reader.__next__()

labels = np.array(
    [
        reader.__next__()[3]
        for index in range(SET_SIZE)
    ],
    dtype = 'float32'
)

# Process data

In [5]:
features, labels = shuffle(features, labels)

In [6]:
def normalize(image_data):
    lower_bound = -0.5
    upper_bound = 0.5
    minimum_pixel = 0
    maximum_pixel = 255
    
    return lower_bound + (
        (image_data - minimum_pixel) *
        (upper_bound - lower_bound) /
        (maximum_pixel - minimum_pixel)
    )

normalized_features = normalize(features)

In [7]:
epsilon = 1e-5

assert math.isclose(
    np.min(normalized_features),
    -0.5,
    abs_tol = epsilon
) and math.isclose(
    np.max(normalized_features),
    0.5,
    abs_tol = epsilon
), 'Range is: {} to {}. It must be -0.5 to 0.5'.format(
    np.min(normalized_features),
    np.max(normalized_features)
)

In [8]:
print('Features shape:', normalized_features.shape)
print('Labels shape:', labels.shape)

Features shape: (256, 160, 320, 3)
Labels shape: (256,)


# Design model

Use model by NVIDIA.

Source: https://arxiv.org/pdf/1604.07316v1.pdf

In [9]:
width = normalized_features.shape[1]
length = normalized_features.shape[2]
depth = normalized_features.shape[3]

In [10]:
# Comment out dimensions from NVIDIA model after testing model output shape
# width = 66
# length = 200
# depth = 3

In [11]:
convolution_filter = 24
kernel_size = 5
stride_size = 2

model = Sequential()
model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 36

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 48

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    subsample = (stride_size, stride_size),
    input_shape = (width, length, depth)
))

convolution_filter = 64
kernel_size = 3

model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    input_shape = (width, length, depth)
))
model.add(Convolution2D(
    convolution_filter,
    kernel_size,
    kernel_size,
    border_mode = 'valid',
    input_shape = (width, length, depth)
))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))
model.add(Activation('softmax'))

print('Shape:', model.output_shape)

Shape: (None, 1)


# Train model

In [ ]:
model.compile(optimizer = 'sgd', loss = 'mse')
history = model.fit(normalized_features, labels, nb_epoch = EPOCH, validation_split = VALIDATION_SET_SIZE)

# Save model

In [ ]:
model.save('model.h5')

In [ ]:
del model

# Load model

In [ ]:
model = load_model('model.h5')

# Evaluate model

# Experiment

1.  Final validation loss = 0.0352
    - Final training loss = 0.0083
    - Model may be overfitting as difference between training and validation loss increases per epoch
    - Loss becomes not a number when training model again
    - Image
        - Center
        - Normalized
    - Set size
        - Training = 204
        - Validation = 52
    - Model
        - Stochastic gradient descent
        - Epoch = 4
        - Longest epoch = 18 s
        - Samples per second = 204 / 18 = 11